In [1]:
import numpy as np
from scipy.stats import ttest_rel
import pandas as pd

In [2]:
store_pvalues = {}
for dataset in ['pereira', 'blank', 'fedorenko']:
    for shuffle_str in ['shuffled', 'contig']:
        for noL2_str in ['_noL2custom', '']:
            results = pd.read_csv(f'/home2/ebrahim/beyond-brainscore/analyze_results/figures_code/figures_data/figure1/{dataset}_pearson_r{noL2_str}_{shuffle_str}.csv')
            for fe in ['', '-mp', '-sp']:
                results_gpt2xl = results.loc[results.Model==f'GPT2-XL{fe}']['perf'].to_numpy()
                results_oasm = results.loc[results.Model=='OASM']['perf'].to_numpy()
                result = ttest_rel(results_gpt2xl, results_oasm)
                if np.mean(results_oasm) > np.mean(results_gpt2xl):
                    store_pvalues[f"{dataset}_{shuffle_str}{noL2_str}_{fe}"] = f"OASM better: {result.pvalue}"
                else:
                    store_pvalues[f"{dataset}_{shuffle_str}{noL2_str}_{fe}"] = f"GPT2XL better: {result.pvalue}"
store_pvalues

{'pereira_shuffled_noL2custom_': 'OASM better: 0.0005829879526030354',
 'pereira_shuffled_noL2custom_-mp': 'OASM better: 8.415461363884617e-06',
 'pereira_shuffled_noL2custom_-sp': 'OASM better: 6.845450682602247e-06',
 'pereira_shuffled_': 'OASM better: 0.0012114104713784794',
 'pereira_shuffled_-mp': 'OASM better: 6.857022634777965e-05',
 'pereira_shuffled_-sp': 'OASM better: 0.00025202230434096434',
 'pereira_contig_noL2custom_': 'GPT2XL better: 0.0006467004109932069',
 'pereira_contig_noL2custom_-mp': 'GPT2XL better: 0.000853685097633342',
 'pereira_contig_noL2custom_-sp': 'GPT2XL better: 0.0009804389292874375',
 'pereira_contig_': 'GPT2XL better: 0.0005507034926934181',
 'pereira_contig_-mp': 'GPT2XL better: 0.00043633780032063205',
 'pereira_contig_-sp': 'GPT2XL better: 0.000749761231008731',
 'blank_shuffled_noL2custom_': 'OASM better: 4.080268110718511e-05',
 'blank_shuffled_noL2custom_-mp': 'OASM better: 5.494779319878125e-05',
 'blank_shuffled_noL2custom_-sp': 'OASM better: 4

In [27]:
def paired_t_test(x, y):
    """
    Compute the t-score for a paired t-test.

    Parameters:
    - x: numpy array of shape (n,), the first set of measurements.
    - y: numpy array of shape (n,), the second set of measurements.

    Returns:
    - t_score: float, the t-score for the paired t-test.
    """
    if len(x) != len(y):
        raise ValueError("Input arrays must have the same length.")
    
    # Calculate the differences
    differences = x - y
    
    # Mean and standard deviation of the differences
    mean_diff = np.mean(differences)
    std_diff = np.std(differences, ddof=1)  # Sample standard deviation (ddof=1)
    
    # Number of samples
    n = len(differences)
    
    # Compute the t-score
    t_score = mean_diff / (std_diff / np.sqrt(n))
    
    return t_score, mean_diff, std_diff

In [31]:
store_pvalues = {}
for dataset in ['pereira', 'blank', 'fedorenko']:
    for fe in ['-lt', '-mp', '-sp']:
        results = pd.read_csv(f'/home2/ebrahim/beyond-brainscore/analyze_results/figures_code/figures_data/figure4/{dataset}_pearson_r.csv')
        results_gpt2xl = results.loc[results.Model==f'GPT2XL{fe}']['perf'].to_numpy()
        results_simple = results.loc[results.Model=='Simple']['perf'].to_numpy()
        result = ttest_rel(results_gpt2xl, results_simple)
        
                    
        if np.mean(results_simple) > np.mean(results_gpt2xl):
            store_pvalues[f"{dataset}_{fe}"] = f"Simple better: {result.pvalue}"
        else:
            store_pvalues[f"{dataset}_{fe}"] = f"GPT2XL better: {result.pvalue}"
            
store_pvalues
    

[0.08346577 0.04533634 0.06363695 0.08628033 0.01109693]
[0.07385403 0.05178941 0.05785515 0.06925373 0.01109693]
[0.08099617 0.05677894 0.04873938 0.07278461 0.01109693]


{'pereira_-lt': 'GPT2XL better: 0.004724223518335061',
 'pereira_-mp': 'GPT2XL better: 0.13243890812810272',
 'pereira_-sp': 'GPT2XL better: 0.014500470367014146',
 'blank_-lt': 'Simple better: 0.013885040925368124',
 'blank_-mp': 'Simple better: 0.00904289156248817',
 'blank_-sp': 'Simple better: 0.01127266136623814',
 'fedorenko_-lt': 'GPT2XL better: 0.7785868321863119',
 'fedorenko_-mp': 'GPT2XL better: 0.5708588256662082',
 'fedorenko_-sp': 'GPT2XL better: 0.520582845367957'}

In [42]:
def compute_frac_sig(feature_extraction_arr, dataset, figure_num):
    
    df = pd.read_csv(f'/home2/ebrahim/beyond-brainscore/analyze_results/figures_code/figures_data/figure{figure_num}/pvalues_{dataset}.csv')
    
    for fe in feature_extraction_arr: 
        
        df_fe = df.loc[df.fe==fe].copy()
        df_fe['pval_sig'] = np.where(df_fe['pval']<0.05, 1, 0)
        df_fe['pval_gpt2xl_sig'] = np.where(df_fe['pval_gpt2xl_sig']<0.05, 1, 0)
        
        print(df_fe.groupby('subject')['pval_gpt2xl_sig'].sum())
        
        df_fe = df_fe.loc[df_fe['pval_gpt2xl_sig'] == 1]
        
        print(df_fe.groupby('subject')['pval_sig'].sum())

        # Calculate the mean proportion of significant p-values per subject
        subject_means = df_fe.groupby('subject')['pval_sig'].mean()
        
    
        # Calculate the overall mean and standard error
        overall_mean = subject_means.mean() * 100  # Convert to percentage
        
        std_error = (subject_means.std() / np.sqrt(len(subject_means))) * 100  # Convert to percentage

        # Print the results with two significant digits
        print(f"{overall_mean:.3g} {std_error:.3g}%")

In [43]:
compute_frac_sig(['-lt', '-mp', '-sp'], 'pereira',2)
compute_frac_sig(['-lt', '-mp', '-sp'], 'fedorenko',2)
compute_frac_sig(['-lt', '-mp', '-sp'], 'blank',2)

11173
2614
21.8 9.1%
10334
2165
18.8 7.94%
10499
2822
24.1 9.86%
52
25
54.3 10.8%
52
30
54.7 9.25%
56
25
39.5 14.3%
2
0
0 nan%
22
0
0 0%
17
0
0 0%


In [9]:
compute_frac_sig(['-lt', '-mp', '-sp'], 'pereira', 4)
compute_frac_sig(['-lt', '-mp', '-sp'], 'fedorenko', 4)
compute_frac_sig(['-lt', '-mp', '-sp'], 'blank', 4)

0.133 0.104%
0.895 0.798%
0.713 0.435%
0 0%
3.78 2.51%
2.5 2.5%
13.3 2.04%
23.3 8.5%
33.3 12.6%


In [6]:
compute_frac_sig(['-lt', '-mp', '-sp'], 'pereira',5)
compute_frac_sig(['-lt', '-mp', '-sp'], 'fedorenko',5)
compute_frac_sig(['-lt', '-mp', '-sp'], 'blank',5)

0 0%
0 0%
0 0%
0 0%
0 0%
0 0%
0 0%
0 0%
0 0%
